In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
input1 = tf.placeholder(tf.int8)
input2 = tf.placeholder(tf.int8)

add = tf.add(input1, input2)
sub = tf.multiply(input1, input2)
result = tf.multiply(add, sub)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(result, feed_dict={input1: 2, input2: 3}))
    writer = tf.summary.FileWriter("./graph",sess.graph)


In [ ]:
import tensorflow.contrib.learn as learn
iris_data,iris_label=learn.datasets.load_iris()
house_data,house_label=learn.datasets.load_boston()

In [ ]:
import tensorflow.examples.tutorials.mnist.input_data

In [ ]:
mnist = tensorflow.examples.tutorials.mnist.input_data.read_data_sets("/home/tao/tao/DeepLearning/MNIST_data", one_hot=True)

In [ ]:
# MNIST EXAMPLE


#设置输入格式
x = tf.placeholder(tf.float32, [None, 784])
y_actual = tf.placeholder(tf.float32, shape=[None, 10])

# 一层全连接，初始化网络参数
W1 = tf.Variable(tf.zeros([784,10]))      
b1 =  tf.Variable(tf.zeros([10]))
# W2 = tf.truncated_normal([100,10], stddev=0.1)  
# b2 =  tf.Variable(tf.zeros([10]))


# hidden
# hidden = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

# 直接softmax分类
y_predict = tf.nn.softmax(tf.matmul(x,W1) + b1) 

# 交叉熵损失
cross_entropy = -tf.reduce_sum(y_actual*tf.log(y_predict))

# 以0.01的学习速率，以交叉熵为优化目标函数，以梯度下降法进行优化
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

#在测试阶段，测试准确度计算
correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_actual,1))   
#多个批次的准确度均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))                

# 初始化全部变量
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(2000):
        # mini batch大小为100
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # 训练数据
        sess.run(train_step, feed_dict={x: batch_xs, y_actual: batch_ys})
        if(i%100==0):                  #每训练100次，测试一次
            print("accuracy:",sess.run(accuracy, feed_dict={x: mnist.test.images, y_actual: mnist.test.labels}))

summary_op = tf.merge_all_summaries()
summary_writer = tf.train.SummaryWriter(FLAGS.train_dir,graph_def=sess.graph_def)
            

In [ ]:
# 权重初始化 使用旧版本API
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# 卷积和池化
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# 输入数据准备
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# 第一层卷积与池化，32个filter
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# reshape中的参数-1表示这一项的具体数值由numpy提供计算
x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 第二层卷积与池化
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Full Connect层
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout层
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Softmax层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %f"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

writer = tf.summary.FileWriter("./graph",sess.graph)
print("test accuracy %f"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


In [ ]:
# 莫烦的博客的例子，学习一根直线
x_data = tf.Variable(tf.random_uniform(dtype=tf.float32,minval=-10,maxval=10,shape=[1,100]),name="x")
y_data = x_data * 0.8 + 1.3 + tf.random_normal(mean=0,stddev=0.1,shape=[1,100])
w = tf.Variable(tf.random_uniform(maxval=1,minval=-1,shape=[1]))
b = tf.Variable(tf.zeros(shape=[1]))
y_predict = w*x_data+b
square_loss = tf.reduce_mean(tf.square(y_data - y_predict))

train_step = tf.train.GradientDescentOptimizer(0.03).minimize(square_loss)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graph",sess.graph)
    sess.run(init)
    for i in range(200):
        if i%10 == 0:
            print("stage%i,loss:%f" % (i, sess.run(square_loss)))
            print("w=%f, b=%f" % (sess.run(w), sess.run(b)))
            print()
        sess.run(train_step)
    

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
x_data = np.linspace(-1,1,300, dtype=np.float32)[:, np.newaxis].T
noise = np.random.normal(0, 0.05, x_data.shape).astype(np.float32)
y_data = np.square(x_data) - 0.5 + noise


with tf.name_scope("input"):
    input_sample = tf.placeholder(tf.float32, [1, None])
    label = tf.placeholder(tf.float32, [1, None])


fig = plt.figure()
plt.ion()

ax = fig.add_subplot(8,5,1)
ax.scatter(x_data[0], y_data[0])


def add_dense_layer(x, input_size, output_size, name, acti=None):
    with tf.name_scope(name):
        with tf.name_scope("W"):
            W = tf.Variable(tf.truncated_normal(mean=0, shape=[output_size, input_size], stddev=0.1))
            tf.summary.histogram(name+"_weight", W)
        with tf.name_scope("b"):
            b = tf.Variable(tf.zeros([output_size,1]) + 0.1)
            tf.summary.histogram(name+"_b", b)
        with tf.name_scope("z"):
            z = tf.matmul(W, x) + b
        if acti:
            output = acti(z)
        else:
            output = z
        tf.summary.histogram(name+"_output", output)
        return output

hidden = add_dense_layer(input_sample, 1, 10, "layer1", tf.nn.relu)
prediction = add_dense_layer(hidden, 10, 1, "layer2")

with tf.name_scope("square_loss"):
    square_loss = tf.reduce_mean(tf.square(prediction - label))
    tf.summary.scalar("square_loss", square_loss)
    
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(square_loss)

merged = tf.summary.merge_all()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("./graph", sess.graph)
    j=1
    for i in range(2000):
        sess.run(train_step, feed_dict={input_sample:x_data, label:y_data})
        if(i % 50 == 0):
            result = sess.run(merged, feed_dict={input_sample:x_data, label:y_data})
            writer.add_summary(result, i)
#             prediction_value = sess.run(prediction, feed_dict={input_sample: x_data})
#             # plot the prediction
#             ax=fig.add_subplot(8,5,j)
#             j+=1
#             ax.plot(x_data[0], prediction_value[0], 'r-', lw=5,)
#             plt.draw()
#             plt.pause(0.1)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.ion()
plt.show()
ax.plot([1,2],[1,1])
for i in range(10):
    print(i)
    ax.plot([1,2],[i,i+1])


In [ ]:


# View more python learning tutorial on my Youtube and Youku channel!!!

# Youtube video tutorial: https://www.youtube.com/channel/UCdyjiB5H8Pu7aDTNVXTTpcg
# Youku video tutorial: http://i.youku.com/pythontutorial

"""
Please note, this code is only for python 3+. If you are using python 2+, please modify the code accordingly.
"""
from __future__ import print_function
import tensorflow as tf
import numpy as np


def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs


# Make up some real data
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))
    tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("logs/", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        result = sess.run(merged,
                          feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result, i)


